In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf
tfd = tf.contrib.distributions 
tfb = tfd.bijectors

/Users/frank/anaconda3/envs/sampling/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [7]:
DEBUG = True

In [11]:
batch_size = 512
x2_dist = tfd.Normal(loc = 0., scale = 4.)
x2_samples = x2_dist.sample(batch_size)
if DEBUG:
    print('x2_samples: ',x2_samples)

x2_samples:  Tensor("Normal_6/sample/Reshape:0", shape=(512,), dtype=float32)


In [22]:
x1_dist = tfd.Normal(loc = 0.25 * tf.square(x2_samples),
                    scale = tf.ones(batch_size, dtype = tf.float32))
x1_samples = x1_dist.sample()
if DEBUG:
    print('x1_samples:',x1_samples)

x1_samples: Tensor("Normal_10/sample/Reshape:0", shape=(512,), dtype=float32)


In [24]:
x_samples = tf.stack([x1_samples, x2_samples], axis = 1)
if DEBUG:
    print('x_samples:', x_samples)

x_samples: Tensor("stack_1:0", shape=(512, 2), dtype=float32)


In [28]:
# Use Isotropic Gaussian

base_dist = tfd.MultivariateNormalDiag(loc = tf.zeros([2], tf.float32))

In [30]:
class LeakyReLU(tfb.Bijector):
    def __init__(self, alpha = 0.5, validate_args = False, name = 'leaky_relu'):
        super(LeakyRelu, self).__init__(
                    event_ndims = 1, validate_args = validate_args, name = name)
        self.alpha = alpha 
    
    def _forward(self, x):
        return tf.where(tf.greater_equal(x,0), x, self.alpha * x)